In [1]:
#setup modules. code in part from ageron and hvass 
import tensorflow as tf
import numpy as np
from functools import partial

from datetime import datetime

#now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
#root_logdir = "/Users/Tim/Development/ConvNN/logs/recoded_cnn"
#logdir = "{}/run-{}/".format(root_logdir,now)

In [2]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

## Dropout rate = 0.1

In [3]:
#derived in part from ageron and hvass
height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_maps = 16
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "SAME"

conv2_maps = 36
conv2_ksize = 3
conv2_stride = 1
conv2_pad = "SAME"

n_fc1 = 128
fc1_dropout_rate = 0.1

n_outputs = 10

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    training = tf.placeholder_with_default(False, shape=[], name="training")
    
conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_maps, kernel_size=conv1_ksize,
                        strides=conv1_stride, padding=conv1_pad, activation=tf.nn.relu, name="conv1")
conv1_maxpool = tf.nn.max_pool(value=conv1, 
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
conv2 = tf.layers.conv2d(conv1_maxpool, filters=conv2_maps, kernel_size=conv2_ksize,
                        strides=conv2_stride, padding=conv2_pad, activation=tf.nn.relu, name="conv2")
conv2_maxpool = tf.nn.max_pool(value=conv2,
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
flat_layer = tf.reshape(conv2_maxpool, shape=[-1, conv2_maps * 7 * 7], name="flat_layer")

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(flat_layer, n_fc1, activation=tf.nn.relu, name="fc1")
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout_rate, training=training)
    
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_prob = tf.nn.softmax(logits, name="Y_prob")
    
    
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()




In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [5]:
def get_model_params():
    gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {gvar.op.name: value for gvar, value in zip(gvars, tf.get_default_session().run(gvars))}

def restore_model_params(model_params):
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: tf.get_default_graph().get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    tf.get_default_session().run(assign_ops, feed_dict=feed_dict)

In [6]:
n_epochs = 1000
batch_size = 100

best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % check_interval == 0:
                loss_val = loss.eval(feed_dict={X: mnist.validation.images,
                                                y: mnist.validation.labels})
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
                                           y: mnist.validation.labels})
        print("Epoch {}, train accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_train * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                        y: mnist.test.labels})
    print("Final accuracy on test set:", acc_test)
    save_path = saver.save(sess, "/home/ec2-user/notebooks/Development/ConvNN/AWS_Models/recoded_cnn_estop.ckpt")

Epoch 0, train accuracy: 99.0000%, valid. accuracy: 97.9600%, valid. best loss: 0.076914
Epoch 1, train accuracy: 98.0000%, valid. accuracy: 98.0400%, valid. best loss: 0.053296
Epoch 2, train accuracy: 99.0000%, valid. accuracy: 98.4800%, valid. best loss: 0.053296
Epoch 3, train accuracy: 98.0000%, valid. accuracy: 98.7200%, valid. best loss: 0.047658
Epoch 4, train accuracy: 100.0000%, valid. accuracy: 99.0800%, valid. best loss: 0.037479
Epoch 5, train accuracy: 99.0000%, valid. accuracy: 98.9400%, valid. best loss: 0.034206
Epoch 6, train accuracy: 100.0000%, valid. accuracy: 99.0600%, valid. best loss: 0.033656
Epoch 7, train accuracy: 100.0000%, valid. accuracy: 99.1400%, valid. best loss: 0.030817
Epoch 8, train accuracy: 100.0000%, valid. accuracy: 99.2000%, valid. best loss: 0.030817
Epoch 9, train accuracy: 100.0000%, valid. accuracy: 99.1400%, valid. best loss: 0.030817
Epoch 10, train accuracy: 100.0000%, valid. accuracy: 99.0600%, valid. best loss: 0.030369
Epoch 11, trai

## Dropout Rate = 0.2

In [7]:
reset_graph()

In [8]:
height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_maps = 16
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "SAME"

conv2_maps = 36
conv2_ksize = 3
conv2_stride = 1
conv2_pad = "SAME"

n_fc1 = 128
fc1_dropout_rate = 0.2

n_outputs = 10

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    training = tf.placeholder_with_default(False, shape=[], name="training")
    
conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_maps, kernel_size=conv1_ksize,
                        strides=conv1_stride, padding=conv1_pad, activation=tf.nn.relu, name="conv1")
conv1_maxpool = tf.nn.max_pool(value=conv1, 
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
conv2 = tf.layers.conv2d(conv1_maxpool, filters=conv2_maps, kernel_size=conv2_ksize,
                        strides=conv2_stride, padding=conv2_pad, activation=tf.nn.relu, name="conv2")
conv2_maxpool = tf.nn.max_pool(value=conv2,
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
flat_layer = tf.reshape(conv2_maxpool, shape=[-1, conv2_maps * 7 * 7], name="flat_layer")

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(flat_layer, n_fc1, activation=tf.nn.relu, name="fc1")
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout_rate, training=training)
    
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_prob = tf.nn.softmax(logits, name="Y_prob")
    
    
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()



In [9]:
n_epochs = 1000
batch_size = 100

best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % check_interval == 0:
                loss_val = loss.eval(feed_dict={X: mnist.validation.images,
                                                y: mnist.validation.labels})
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
                                           y: mnist.validation.labels})
        print("Epoch {}, train accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_train * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                        y: mnist.test.labels})
    print("Final accuracy on test set:", acc_test)
    save_path = saver.save(sess, "/home/ec2-user/notebooks/Development/ConvNN/AWS_Models/recoded_cnn_estop_2.ckpt")


Epoch 0, train accuracy: 97.0000%, valid. accuracy: 97.9000%, valid. best loss: 0.072068
Epoch 1, train accuracy: 98.0000%, valid. accuracy: 98.4400%, valid. best loss: 0.054268
Epoch 2, train accuracy: 99.0000%, valid. accuracy: 98.7000%, valid. best loss: 0.049214
Epoch 3, train accuracy: 100.0000%, valid. accuracy: 99.0400%, valid. best loss: 0.041289
Epoch 4, train accuracy: 98.0000%, valid. accuracy: 99.0200%, valid. best loss: 0.034344
Epoch 5, train accuracy: 100.0000%, valid. accuracy: 99.1600%, valid. best loss: 0.034344
Epoch 6, train accuracy: 99.0000%, valid. accuracy: 99.1200%, valid. best loss: 0.029947
Epoch 7, train accuracy: 100.0000%, valid. accuracy: 98.6600%, valid. best loss: 0.029947
Epoch 8, train accuracy: 100.0000%, valid. accuracy: 99.0800%, valid. best loss: 0.029947
Epoch 9, train accuracy: 100.0000%, valid. accuracy: 99.1600%, valid. best loss: 0.029947
Epoch 10, train accuracy: 100.0000%, valid. accuracy: 99.1600%, valid. best loss: 0.029947
Epoch 11, trai

## Dropout Rate = 0.3

In [10]:
reset_graph()
height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_maps = 16
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "SAME"

conv2_maps = 36
conv2_ksize = 3
conv2_stride = 1
conv2_pad = "SAME"

n_fc1 = 128
fc1_dropout_rate = 0.3

n_outputs = 10

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    training = tf.placeholder_with_default(False, shape=[], name="training")
    
conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_maps, kernel_size=conv1_ksize,
                        strides=conv1_stride, padding=conv1_pad, activation=tf.nn.relu, name="conv1")
conv1_maxpool = tf.nn.max_pool(value=conv1, 
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
conv2 = tf.layers.conv2d(conv1_maxpool, filters=conv2_maps, kernel_size=conv2_ksize,
                        strides=conv2_stride, padding=conv2_pad, activation=tf.nn.relu, name="conv2")
conv2_maxpool = tf.nn.max_pool(value=conv2,
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
flat_layer = tf.reshape(conv2_maxpool, shape=[-1, conv2_maps * 7 * 7], name="flat_layer")

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(flat_layer, n_fc1, activation=tf.nn.relu, name="fc1")
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout_rate, training=training)
    
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_prob = tf.nn.softmax(logits, name="Y_prob")
    
    
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()



In [11]:
n_epochs = 1000
batch_size = 100

best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % check_interval == 0:
                loss_val = loss.eval(feed_dict={X: mnist.validation.images,
                                                y: mnist.validation.labels})
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
                                           y: mnist.validation.labels})
        print("Epoch {}, train accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_train * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                        y: mnist.test.labels})
    print("Final accuracy on test set:", acc_test)
    save_path = saver.save(sess, "/home/ec2-user/notebooks/Development/ConvNN/AWS_Models/recoded_cnn_estop_3.ckpt")


Epoch 0, train accuracy: 100.0000%, valid. accuracy: 98.1800%, valid. best loss: 0.065674
Epoch 1, train accuracy: 99.0000%, valid. accuracy: 98.2400%, valid. best loss: 0.050099
Epoch 2, train accuracy: 100.0000%, valid. accuracy: 98.3200%, valid. best loss: 0.049980
Epoch 3, train accuracy: 99.0000%, valid. accuracy: 99.0200%, valid. best loss: 0.046525
Epoch 4, train accuracy: 98.0000%, valid. accuracy: 98.4200%, valid. best loss: 0.034817
Epoch 5, train accuracy: 99.0000%, valid. accuracy: 99.0200%, valid. best loss: 0.028693
Epoch 6, train accuracy: 100.0000%, valid. accuracy: 98.9200%, valid. best loss: 0.028693
Epoch 7, train accuracy: 100.0000%, valid. accuracy: 99.1800%, valid. best loss: 0.028693
Epoch 8, train accuracy: 100.0000%, valid. accuracy: 99.1200%, valid. best loss: 0.028693
Epoch 9, train accuracy: 99.0000%, valid. accuracy: 99.2600%, valid. best loss: 0.026986
Epoch 10, train accuracy: 100.0000%, valid. accuracy: 98.8600%, valid. best loss: 0.025961
Epoch 11, trai

## Dropout Rate = 0.4

In [12]:
reset_graph()
height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_maps = 16
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "SAME"

conv2_maps = 36
conv2_ksize = 3
conv2_stride = 1
conv2_pad = "SAME"

n_fc1 = 128
fc1_dropout_rate = 0.4

n_outputs = 10

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    training = tf.placeholder_with_default(False, shape=[], name="training")
    
conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_maps, kernel_size=conv1_ksize,
                        strides=conv1_stride, padding=conv1_pad, activation=tf.nn.relu, name="conv1")
conv1_maxpool = tf.nn.max_pool(value=conv1, 
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
conv2 = tf.layers.conv2d(conv1_maxpool, filters=conv2_maps, kernel_size=conv2_ksize,
                        strides=conv2_stride, padding=conv2_pad, activation=tf.nn.relu, name="conv2")
conv2_maxpool = tf.nn.max_pool(value=conv2,
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
flat_layer = tf.reshape(conv2_maxpool, shape=[-1, conv2_maps * 7 * 7], name="flat_layer")

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(flat_layer, n_fc1, activation=tf.nn.relu, name="fc1")
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout_rate, training=training)
    
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_prob = tf.nn.softmax(logits, name="Y_prob")
    
    
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()



In [13]:
n_epochs = 1000
batch_size = 100

best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % check_interval == 0:
                loss_val = loss.eval(feed_dict={X: mnist.validation.images,
                                                y: mnist.validation.labels})
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
                                           y: mnist.validation.labels})
        print("Epoch {}, train accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_train * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                        y: mnist.test.labels})
    print("Final accuracy on test set:", acc_test)
    save_path = saver.save(sess, "/home/ec2-user/notebooks/Development/ConvNN/AWS_Models/recoded_cnn_estop_4.ckpt")


Epoch 0, train accuracy: 98.0000%, valid. accuracy: 97.9400%, valid. best loss: 0.075180
Epoch 1, train accuracy: 98.0000%, valid. accuracy: 98.5600%, valid. best loss: 0.050456
Epoch 2, train accuracy: 100.0000%, valid. accuracy: 98.9000%, valid. best loss: 0.040040
Epoch 3, train accuracy: 100.0000%, valid. accuracy: 98.9800%, valid. best loss: 0.036179
Epoch 4, train accuracy: 99.0000%, valid. accuracy: 98.8600%, valid. best loss: 0.033394
Epoch 5, train accuracy: 100.0000%, valid. accuracy: 99.0800%, valid. best loss: 0.033394
Epoch 6, train accuracy: 100.0000%, valid. accuracy: 99.0800%, valid. best loss: 0.028342
Epoch 7, train accuracy: 99.0000%, valid. accuracy: 99.0200%, valid. best loss: 0.028342
Epoch 8, train accuracy: 100.0000%, valid. accuracy: 99.1400%, valid. best loss: 0.028342
Epoch 9, train accuracy: 100.0000%, valid. accuracy: 98.9400%, valid. best loss: 0.028342
Epoch 10, train accuracy: 100.0000%, valid. accuracy: 99.1800%, valid. best loss: 0.028342
Epoch 11, tra

## Dropout Rate = 0.5

In [14]:
reset_graph()
height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_maps = 16
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "SAME"

conv2_maps = 36
conv2_ksize = 3
conv2_stride = 1
conv2_pad = "SAME"

n_fc1 = 128
fc1_dropout_rate = 0.5

n_outputs = 10

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    training = tf.placeholder_with_default(False, shape=[], name="training")
    
conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_maps, kernel_size=conv1_ksize,
                        strides=conv1_stride, padding=conv1_pad, activation=tf.nn.relu, name="conv1")
conv1_maxpool = tf.nn.max_pool(value=conv1, 
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
conv2 = tf.layers.conv2d(conv1_maxpool, filters=conv2_maps, kernel_size=conv2_ksize,
                        strides=conv2_stride, padding=conv2_pad, activation=tf.nn.relu, name="conv2")
conv2_maxpool = tf.nn.max_pool(value=conv2,
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
flat_layer = tf.reshape(conv2_maxpool, shape=[-1, conv2_maps * 7 * 7], name="flat_layer")

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(flat_layer, n_fc1, activation=tf.nn.relu, name="fc1")
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout_rate, training=training)
    
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_prob = tf.nn.softmax(logits, name="Y_prob")
    
    
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()



In [15]:
n_epochs = 1000
batch_size = 100

best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % check_interval == 0:
                loss_val = loss.eval(feed_dict={X: mnist.validation.images,
                                                y: mnist.validation.labels})
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
                                           y: mnist.validation.labels})
        print("Epoch {}, train accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_train * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                        y: mnist.test.labels})
    print("Final accuracy on test set:", acc_test)
    save_path = saver.save(sess, "/home/ec2-user/notebooks/Development/ConvNN/AWS_Models/recoded_cnn_estop_5.ckpt")


Epoch 0, train accuracy: 98.0000%, valid. accuracy: 98.1400%, valid. best loss: 0.069863
Epoch 1, train accuracy: 96.0000%, valid. accuracy: 98.7200%, valid. best loss: 0.058636
Epoch 2, train accuracy: 99.0000%, valid. accuracy: 98.7400%, valid. best loss: 0.044137
Epoch 3, train accuracy: 99.0000%, valid. accuracy: 99.0400%, valid. best loss: 0.039669
Epoch 4, train accuracy: 100.0000%, valid. accuracy: 98.8000%, valid. best loss: 0.036725
Epoch 5, train accuracy: 100.0000%, valid. accuracy: 98.7800%, valid. best loss: 0.035098
Epoch 6, train accuracy: 99.0000%, valid. accuracy: 98.9400%, valid. best loss: 0.035098
Epoch 7, train accuracy: 100.0000%, valid. accuracy: 98.9800%, valid. best loss: 0.035098
Epoch 8, train accuracy: 99.0000%, valid. accuracy: 99.1000%, valid. best loss: 0.032840
Epoch 9, train accuracy: 100.0000%, valid. accuracy: 98.8400%, valid. best loss: 0.031957
Epoch 10, train accuracy: 100.0000%, valid. accuracy: 99.1600%, valid. best loss: 0.031957
Epoch 11, train

## Dropout Rate = 0.6

In [16]:
reset_graph()
height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_maps = 16
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "SAME"

conv2_maps = 36
conv2_ksize = 3
conv2_stride = 1
conv2_pad = "SAME"

n_fc1 = 128
fc1_dropout_rate = 0.6

n_outputs = 10

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    training = tf.placeholder_with_default(False, shape=[], name="training")
    
conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_maps, kernel_size=conv1_ksize,
                        strides=conv1_stride, padding=conv1_pad, activation=tf.nn.relu, name="conv1")
conv1_maxpool = tf.nn.max_pool(value=conv1, 
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
conv2 = tf.layers.conv2d(conv1_maxpool, filters=conv2_maps, kernel_size=conv2_ksize,
                        strides=conv2_stride, padding=conv2_pad, activation=tf.nn.relu, name="conv2")
conv2_maxpool = tf.nn.max_pool(value=conv2,
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
flat_layer = tf.reshape(conv2_maxpool, shape=[-1, conv2_maps * 7 * 7], name="flat_layer")

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(flat_layer, n_fc1, activation=tf.nn.relu, name="fc1")
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout_rate, training=training)
    
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_prob = tf.nn.softmax(logits, name="Y_prob")
    
    
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()



In [17]:
n_epochs = 1000
batch_size = 100

best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % check_interval == 0:
                loss_val = loss.eval(feed_dict={X: mnist.validation.images,
                                                y: mnist.validation.labels})
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
                                           y: mnist.validation.labels})
        print("Epoch {}, train accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_train * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                        y: mnist.test.labels})
    print("Final accuracy on test set:", acc_test)
    save_path = saver.save(sess, "/home/ec2-user/notebooks/Development/ConvNN/AWS_Models/recoded_cnn_estop_6.ckpt")


Epoch 0, train accuracy: 95.0000%, valid. accuracy: 98.1600%, valid. best loss: 0.077152
Epoch 1, train accuracy: 100.0000%, valid. accuracy: 98.8000%, valid. best loss: 0.050288
Epoch 2, train accuracy: 98.0000%, valid. accuracy: 98.5000%, valid. best loss: 0.046833
Epoch 3, train accuracy: 99.0000%, valid. accuracy: 98.9800%, valid. best loss: 0.042262
Epoch 4, train accuracy: 100.0000%, valid. accuracy: 99.1000%, valid. best loss: 0.034267
Epoch 5, train accuracy: 99.0000%, valid. accuracy: 98.7200%, valid. best loss: 0.032983
Epoch 6, train accuracy: 100.0000%, valid. accuracy: 98.9400%, valid. best loss: 0.032983
Epoch 7, train accuracy: 99.0000%, valid. accuracy: 99.0800%, valid. best loss: 0.032983
Epoch 8, train accuracy: 99.0000%, valid. accuracy: 99.1000%, valid. best loss: 0.030722
Epoch 9, train accuracy: 100.0000%, valid. accuracy: 98.7600%, valid. best loss: 0.030722
Epoch 10, train accuracy: 100.0000%, valid. accuracy: 99.1400%, valid. best loss: 0.030722
Epoch 11, train

## Dropout Rate = 0.7

In [18]:
reset_graph()
height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_maps = 16
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "SAME"

conv2_maps = 36
conv2_ksize = 3
conv2_stride = 1
conv2_pad = "SAME"

n_fc1 = 128
fc1_dropout_rate = 0.7

n_outputs = 10

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    training = tf.placeholder_with_default(False, shape=[], name="training")
    
conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_maps, kernel_size=conv1_ksize,
                        strides=conv1_stride, padding=conv1_pad, activation=tf.nn.relu, name="conv1")
conv1_maxpool = tf.nn.max_pool(value=conv1, 
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
conv2 = tf.layers.conv2d(conv1_maxpool, filters=conv2_maps, kernel_size=conv2_ksize,
                        strides=conv2_stride, padding=conv2_pad, activation=tf.nn.relu, name="conv2")
conv2_maxpool = tf.nn.max_pool(value=conv2,
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
flat_layer = tf.reshape(conv2_maxpool, shape=[-1, conv2_maps * 7 * 7], name="flat_layer")

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(flat_layer, n_fc1, activation=tf.nn.relu, name="fc1")
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout_rate, training=training)
    
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_prob = tf.nn.softmax(logits, name="Y_prob")
    
    
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()



In [19]:
n_epochs = 1000
batch_size = 100

best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % check_interval == 0:
                loss_val = loss.eval(feed_dict={X: mnist.validation.images,
                                                y: mnist.validation.labels})
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
                                           y: mnist.validation.labels})
        print("Epoch {}, train accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_train * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                        y: mnist.test.labels})
    print("Final accuracy on test set:", acc_test)
    save_path = saver.save(sess, "/home/ec2-user/notebooks/Development/ConvNN/AWS_Models/recoded_cnn_estop_7.ckpt")


Epoch 0, train accuracy: 100.0000%, valid. accuracy: 97.8400%, valid. best loss: 0.072232
Epoch 1, train accuracy: 100.0000%, valid. accuracy: 98.8000%, valid. best loss: 0.050188
Epoch 2, train accuracy: 100.0000%, valid. accuracy: 98.8400%, valid. best loss: 0.047292
Epoch 3, train accuracy: 100.0000%, valid. accuracy: 98.9800%, valid. best loss: 0.041474
Epoch 4, train accuracy: 99.0000%, valid. accuracy: 98.8800%, valid. best loss: 0.034089
Epoch 5, train accuracy: 100.0000%, valid. accuracy: 99.0200%, valid. best loss: 0.034089
Epoch 6, train accuracy: 99.0000%, valid. accuracy: 99.1200%, valid. best loss: 0.033903
Epoch 7, train accuracy: 100.0000%, valid. accuracy: 99.1600%, valid. best loss: 0.029855
Epoch 8, train accuracy: 100.0000%, valid. accuracy: 99.1200%, valid. best loss: 0.029855
Epoch 9, train accuracy: 100.0000%, valid. accuracy: 98.8200%, valid. best loss: 0.029855
Epoch 10, train accuracy: 100.0000%, valid. accuracy: 99.0600%, valid. best loss: 0.029855
Epoch 11, t

## Dropout Rate = 0.8

In [20]:
reset_graph()
height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_maps = 16
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "SAME"

conv2_maps = 36
conv2_ksize = 3
conv2_stride = 1
conv2_pad = "SAME"

n_fc1 = 128
fc1_dropout_rate = 0.8

n_outputs = 10

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    training = tf.placeholder_with_default(False, shape=[], name="training")
    
conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_maps, kernel_size=conv1_ksize,
                        strides=conv1_stride, padding=conv1_pad, activation=tf.nn.relu, name="conv1")
conv1_maxpool = tf.nn.max_pool(value=conv1, 
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
conv2 = tf.layers.conv2d(conv1_maxpool, filters=conv2_maps, kernel_size=conv2_ksize,
                        strides=conv2_stride, padding=conv2_pad, activation=tf.nn.relu, name="conv2")
conv2_maxpool = tf.nn.max_pool(value=conv2,
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
flat_layer = tf.reshape(conv2_maxpool, shape=[-1, conv2_maps * 7 * 7], name="flat_layer")

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(flat_layer, n_fc1, activation=tf.nn.relu, name="fc1")
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout_rate, training=training)
    
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_prob = tf.nn.softmax(logits, name="Y_prob")
    
    
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()



In [21]:
n_epochs = 1000
batch_size = 100

best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % check_interval == 0:
                loss_val = loss.eval(feed_dict={X: mnist.validation.images,
                                                y: mnist.validation.labels})
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
                                           y: mnist.validation.labels})
        print("Epoch {}, train accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_train * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                        y: mnist.test.labels})
    print("Final accuracy on test set:", acc_test)
    save_path = saver.save(sess, "/home/ec2-user/notebooks/Development/ConvNN/AWS_Models/recoded_cnn_estop_8.ckpt")


Epoch 0, train accuracy: 98.0000%, valid. accuracy: 97.6000%, valid. best loss: 0.073572
Epoch 1, train accuracy: 99.0000%, valid. accuracy: 98.5000%, valid. best loss: 0.060165
Epoch 2, train accuracy: 100.0000%, valid. accuracy: 98.6800%, valid. best loss: 0.042923
Epoch 3, train accuracy: 100.0000%, valid. accuracy: 98.9400%, valid. best loss: 0.037442
Epoch 4, train accuracy: 100.0000%, valid. accuracy: 98.8800%, valid. best loss: 0.036909
Epoch 5, train accuracy: 100.0000%, valid. accuracy: 99.0600%, valid. best loss: 0.033526
Epoch 6, train accuracy: 100.0000%, valid. accuracy: 99.0800%, valid. best loss: 0.031364
Epoch 7, train accuracy: 100.0000%, valid. accuracy: 99.0800%, valid. best loss: 0.031364
Epoch 8, train accuracy: 100.0000%, valid. accuracy: 99.0000%, valid. best loss: 0.031364
Epoch 9, train accuracy: 99.0000%, valid. accuracy: 99.0000%, valid. best loss: 0.031364
Epoch 10, train accuracy: 100.0000%, valid. accuracy: 98.8600%, valid. best loss: 0.031364
Epoch 11, tr

## Dropout Rate = 0.9

In [22]:
reset_graph()
height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_maps = 16
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "SAME"

conv2_maps = 36
conv2_ksize = 3
conv2_stride = 1
conv2_pad = "SAME"

n_fc1 = 128
fc1_dropout_rate = 0.9

n_outputs = 10

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    training = tf.placeholder_with_default(False, shape=[], name="training")
    
conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_maps, kernel_size=conv1_ksize,
                        strides=conv1_stride, padding=conv1_pad, activation=tf.nn.relu, name="conv1")
conv1_maxpool = tf.nn.max_pool(value=conv1, 
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
conv2 = tf.layers.conv2d(conv1_maxpool, filters=conv2_maps, kernel_size=conv2_ksize,
                        strides=conv2_stride, padding=conv2_pad, activation=tf.nn.relu, name="conv2")
conv2_maxpool = tf.nn.max_pool(value=conv2,
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
flat_layer = tf.reshape(conv2_maxpool, shape=[-1, conv2_maps * 7 * 7], name="flat_layer")

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(flat_layer, n_fc1, activation=tf.nn.relu, name="fc1")
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout_rate, training=training)
    
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_prob = tf.nn.softmax(logits, name="Y_prob")
    
    
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()



In [23]:
n_epochs = 1000
batch_size = 100

best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % check_interval == 0:
                loss_val = loss.eval(feed_dict={X: mnist.validation.images,
                                                y: mnist.validation.labels})
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
                                           y: mnist.validation.labels})
        print("Epoch {}, train accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_train * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                        y: mnist.test.labels})
    print("Final accuracy on test set:", acc_test)
    save_path = saver.save(sess, "/home/ec2-user/notebooks/Development/ConvNN/AWS_Models/recoded_cnn_estop_9.ckpt")


Epoch 0, train accuracy: 98.0000%, valid. accuracy: 97.7200%, valid. best loss: 0.069640
Epoch 1, train accuracy: 99.0000%, valid. accuracy: 98.3400%, valid. best loss: 0.060528
Epoch 2, train accuracy: 100.0000%, valid. accuracy: 98.8200%, valid. best loss: 0.044568
Epoch 3, train accuracy: 100.0000%, valid. accuracy: 99.0200%, valid. best loss: 0.039706
Epoch 4, train accuracy: 100.0000%, valid. accuracy: 98.9000%, valid. best loss: 0.035160
Epoch 5, train accuracy: 100.0000%, valid. accuracy: 99.0200%, valid. best loss: 0.033190
Epoch 6, train accuracy: 100.0000%, valid. accuracy: 99.0800%, valid. best loss: 0.033190
Epoch 7, train accuracy: 100.0000%, valid. accuracy: 99.1800%, valid. best loss: 0.033190
Epoch 8, train accuracy: 100.0000%, valid. accuracy: 99.1200%, valid. best loss: 0.032875
Epoch 9, train accuracy: 100.0000%, valid. accuracy: 99.0800%, valid. best loss: 0.032875
Epoch 10, train accuracy: 100.0000%, valid. accuracy: 99.0200%, valid. best loss: 0.032875
Epoch 11, t

## Dropout Rate = 0.0

In [24]:
reset_graph()
height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_maps = 16
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "SAME"

conv2_maps = 36
conv2_ksize = 3
conv2_stride = 1
conv2_pad = "SAME"

n_fc1 = 128
fc1_dropout_rate = 0.0

n_outputs = 10

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    training = tf.placeholder_with_default(False, shape=[], name="training")
    
conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_maps, kernel_size=conv1_ksize,
                        strides=conv1_stride, padding=conv1_pad, activation=tf.nn.relu, name="conv1")
conv1_maxpool = tf.nn.max_pool(value=conv1, 
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
conv2 = tf.layers.conv2d(conv1_maxpool, filters=conv2_maps, kernel_size=conv2_ksize,
                        strides=conv2_stride, padding=conv2_pad, activation=tf.nn.relu, name="conv2")
conv2_maxpool = tf.nn.max_pool(value=conv2,
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
flat_layer = tf.reshape(conv2_maxpool, shape=[-1, conv2_maps * 7 * 7], name="flat_layer")

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(flat_layer, n_fc1, activation=tf.nn.relu, name="fc1")
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout_rate, training=training)
    
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_prob = tf.nn.softmax(logits, name="Y_prob")
    
    
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()



In [25]:
n_epochs = 1000
batch_size = 100

best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % check_interval == 0:
                loss_val = loss.eval(feed_dict={X: mnist.validation.images,
                                                y: mnist.validation.labels})
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
                                           y: mnist.validation.labels})
        print("Epoch {}, train accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_train * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                        y: mnist.test.labels})
    print("Final accuracy on test set:", acc_test)
    save_path = saver.save(sess, "/home/ec2-user/notebooks/Development/ConvNN/AWS_Models/recoded_cnn_estop_0.ckpt")


Epoch 0, train accuracy: 97.0000%, valid. accuracy: 97.8600%, valid. best loss: 0.067926
Epoch 1, train accuracy: 97.0000%, valid. accuracy: 98.7600%, valid. best loss: 0.045842
Epoch 2, train accuracy: 97.0000%, valid. accuracy: 98.7400%, valid. best loss: 0.044538
Epoch 3, train accuracy: 100.0000%, valid. accuracy: 98.9600%, valid. best loss: 0.042487
Epoch 4, train accuracy: 100.0000%, valid. accuracy: 99.0200%, valid. best loss: 0.038078
Epoch 5, train accuracy: 99.0000%, valid. accuracy: 98.8600%, valid. best loss: 0.033953
Epoch 6, train accuracy: 100.0000%, valid. accuracy: 98.7200%, valid. best loss: 0.033953
Epoch 7, train accuracy: 100.0000%, valid. accuracy: 99.1800%, valid. best loss: 0.033953
Epoch 8, train accuracy: 100.0000%, valid. accuracy: 99.1200%, valid. best loss: 0.033177
Epoch 9, train accuracy: 100.0000%, valid. accuracy: 99.1800%, valid. best loss: 0.030592
Epoch 10, train accuracy: 100.0000%, valid. accuracy: 99.0600%, valid. best loss: 0.030592
Epoch 11, tra